In [43]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, LeakyReLU, Conv2DTranspose
from keras.optimizers import Adam
from tensorflow.keras import layers

In [61]:
generator = Sequential()
generator.add(Dense(8*8*48, use_bias=False, input_shape=(100,)))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

generator.add(Reshape((8, 8, 48)))

generator.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

generator.add(Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same', use_bias=False))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

generator.add(Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

discriminator = Sequential()
discriminator.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=(64, 64, 3)))
discriminator.add(LeakyReLU())

discriminator.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU())

discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

gan = Sequential()
gan.add(generator)
gan.add(discriminator)

In [60]:
discriminator = Sequential()
discriminator.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=(64, 64, 3)))
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Conv2D(256, (4, 4), strides=(2, 2), padding='same'))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))


In [62]:
# d_output_shape = (64, 64, 1)
# d_output = discriminator.layers[-2].output
# d_output = Reshape(d_output_shape)(d_output)
# # Define the GAN as a sequential model consisting of the generator followed by the discriminator
# # Define the GAN model
# gan_input = Input(shape=(100,))
# gan_output = generator(gan_input)
# gan_output = discriminator(gan_output)
# gan_output = Concatenate()([gan_output, gan_input])
# gan = Model(gan_input, gan_output)

# gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# gan = Sequential()
# gan.add(generator)
# gan.add(discriminator)

# compile GAN model
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))


In [63]:
# Define the labels and corresponding directories
labels = ['Abdomen and Chest', 'Ankle', 'Cervical Spine','Elbow','Feet','Finger','Forearm','Hand','Hand and Wrist','Hip','Knee','Lower Leg','Lumbar Spine','Lumbar Spine and Thoracic Spine','Others','Pelvis','Shoulder','Sinus','Skull','Thoracic Spine','Wrist']
dirs = ['E:/datanew/Abdomen and Chest','E:/datanew/Ankle','E:/datanew/Cervical Spine','E:/datanew/Elbow','E:/datanew/Feet','E:/datanew/Finger','E:/datanew/Forearm','E:/datanew/Hand','E:/datanew/Hand and Wrist','E:/datanew/Hip','E:/datanew/Knee','E:/datanew/Lower Leg','E:/datanew/Lumbar Spine','E:/datanew/Lumbar Spine and Thoracic Spine','E:/datanew/Others','E:/datanew/Pelvis','E:/datanew/Shoulder','E:/datanew/Sinus','E:/datanew/Skull','E:/datanew/Thoracic Spine','E:/datanew/Wrist', ]

# Define the image data generator
datagen = ImageDataGenerator(rescale=1./255)

# Load the dataset
dataset = datagen.flow_from_directory('E:/datanew', target_size=(256, 256), class_mode=None, batch_size=64)

Found 882 images belonging to 21 classes.


In [64]:
# Train the GAN
for epoch in range(100):
    print(f'Epoch {epoch}')
    
    for batch in dataset:
        # Generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_images = generator.predict(noise)

        # Concatenate the real and fake images
        combined_images = np.concatenate((batch, fake_images))

        # Assign labels to the real and fake images
        labels = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))))

        # Add noise to the labels
        labels += 0.05 * np.random.random(labels.shape)

        # Train the discriminator on the combined images and labels
        discriminator_loss = discriminator.train_on_batch(combined_images, labels)

        # Generate a batch of noise vectors for the generator
        noise = np.random.normal(0, 1, (batch_size, 100))

        # Assign the labels for the fake images as ones
        misleading_targets = np.ones((batch_size, 1))

        # Train the generator on the noise with the misleading targets
        gan_loss = gan.train_on_batch(noise, misleading_targets)

    # Save the generator weights every 10 epochs
    if epoch % 10 == 0:
        generator.save_weights(f'generator_weights_epoch_{epoch}.h5')

    # Generate and save new images for each label
    for label, dir in zip(labels, dirs):
        os.makedirs(dir, exist_ok=True)
        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_images = generator.predict(noise)
        for i in range(batch_size):
            img = array_to_img(fake_images[i] * 255., scale=False)
            img.save(os.path.join(dir, f'{label}_{i}.png'))

Epoch 0
4/4 [==============================] - 0s 19ms/step


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 256 and the array at index 1 has size 64